In [117]:
import coffea
from coffea import util
import numpy as np
import pandas as pd
import itertools
import os

In [30]:
# analysis categories #
ttagcats = ["AT&Pt", "at", "pret", "0t", "1t", ">=1t", "2t", ">=0t"] 
btagcats = ["0b", "1b", "2b"]
ycats = ['cen', 'fwd']

anacats = [ t+b+y for t,b,y in itertools.product( ttagcats, btagcats, ycats) ]
label_dict = {i: label for i, label in enumerate(anacats)}
label_to_int_dict = {label: i for i, label in enumerate(anacats)}

In [166]:
# label_dict

In [151]:
save_csv_filename = 'mistag_rate.csv'

In [218]:
coffea_dir = '../outputs/'
coffeaFiles = {
    "JetHT":{
        "2016APV": {
            "B": coffea_dir+'JetHT_2016APVB.coffea',
            "C": coffea_dir+'JetHT_2016APVC.coffea',
            "D": coffea_dir+'JetHT_2016APVD.coffea',
            "E": coffea_dir+'JetHT_2016APVE.coffea',
        },
        "2016": {
            "F": coffea_dir+'JetHT_2016F.coffea',
            "G": coffea_dir+'JetHT_2016G.coffea',
            "H": coffea_dir+'JetHT_2016H.coffea',
        },
        "2017": '',
        "2018": ''
    },
    
    "TTbar": {
        "2016APV": {
            "700to1000": coffea_dir+'TTbar_2016APV_700to_1000.coffea',
            "1000toInf": coffea_dir+'TTbar_2016APV_1000toInf.coffea',
        },
        "2016": {
            "700to1000": coffea_dir+'TTbar_2016_700to_1000.coffea',
            "1000toInf": coffea_dir+'TTbar_2016_1000toInf.coffea',
        },
        "2017": {
            "700to1000": '',
            "1000toInf": '',
        },
        "2018": {
            "700to1000": '',
            "1000toInf": '',
        }
    }
}




In [219]:
# pbins = numerator.axes['jetp'].centers - numerator.axes['jetp'].widths/2
pbins = np.array([ 400.,  500.,  600.,  800., 1000., 1500., 2000., 3000., 7000.])

In [223]:
IOVs = ['2016']


for IOV in IOVs:
    save_csv_filename = f'mistag_rate_{IOV}.csv'

    jetht_files = []

    for era, file in coffeaFiles['JetHT'][IOV].items():
        if os.path.isfile(file):
            print('loading', file)
            jetht_files.append(util.load(file))
            
            
    ttbar_700to1000 = util.load(coffeaFiles['TTbar'][IOV]['700to1000'])
    ttbar_1000toInf = util.load(coffeaFiles['TTbar'][IOV]['1000toInf'])
    print('loading', coffeaFiles['TTbar'][IOV]['700to1000'])
    print('loading', coffeaFiles['TTbar'][IOV]['1000toInf'])




    mistag_rate_dict = {}
    mistag_rate_dict["jetp bins"] = pbins

    for i, label in label_dict.items():

        jetht_numerator   = jetht_files[0]['numerator'][{'dataset':sum, 'anacat':i}].values()
        jetht_denominator = jetht_files[0]['denominator'][{'dataset':sum, 'anacat':i}].values()

        for file in jetht_files[1:]:
            jetht_numerator   += file['numerator'][{'dataset':sum, 'anacat':i}].values()
            jetht_denominator += file['denominator'][{'dataset':sum, 'anacat':i}].values()

        ttbar_numerator   = ttbar_700to1000['numerator'][{'dataset':sum, 'anacat':i}].values() * ttbar_SF['700to1000']
        ttbar_denominator = ttbar_700to1000['denominator'][{'dataset':sum, 'anacat':i}].values() * ttbar_SF['700to1000']

        ttbar_numerator   += ttbar_1000toInf['numerator'][{'dataset':sum, 'anacat':i}].values() * ttbar_SF['1000toInf']
        ttbar_denominator += ttbar_1000toInf['denominator'][{'dataset':sum, 'anacat':i}].values() * ttbar_SF['1000toInf']

        mistag_rate = (jetht_numerator - ttbar_numerator) / (jetht_denominator - ttbar_denominator)
#         mistag_rate = (jetht_numerator - ttbar_numerator) / (jetht_denominator)

        mistag_rate[np.isnan(mistag_rate)] = 0.

        mistag_rate_dict[label] = mistag_rate
        
    df_mistag = pd.DataFrame(data=mistag_rate_dict)
    df_mistag.to_csv(save_csv_filename)
    print('saving', save_csv_filename)

        

loading ../outputs/JetHT_2016F.coffea
loading ../outputs/JetHT_2016G.coffea
loading ../outputs/JetHT_2016H.coffea
loading ../outputs/TTbar_2016_700to_1000.coffea
loading ../outputs/TTbar_2016_1000toInf.coffea
saving mistag_rate_2016.csv


/tmp/ipykernel_33/2015335619.py:41: RuntimeWarning: invalid value encountered in divide
  mistag_rate = (jetht_numerator - ttbar_numerator) / (jetht_denominator - ttbar_denominator)


In [224]:
df_mistag = pd.DataFrame(data=mistag_rate_dict)

In [225]:
df_mistag[df_mistag.columns[7:13]]

,at0bcen,at0bfwd,at1bcen,at1bfwd,at2bcen,at2bfwd
0,0.049703,0.056627,0.069703,0.063222,0.123204,0.048513
1,0.050615,0.051627,0.075497,0.069642,0.108504,0.091166
2,0.042909,0.043783,0.056891,0.058130,0.085656,0.060743
3,0.038261,0.036193,0.044815,0.046783,0.069295,0.063532
4,0.032559,0.031492,0.034966,0.036909,0.066019,0.054593
5,0.025565,0.024042,0.029679,0.031799,0.036520,0.057750
6,0.028147,0.025961,0.027467,0.034490,0.077573,0.070781
7,0.007234,0.027039,-0.001462,0.034990,-0.016180,-0.004996
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [99]:
np.where(ttbar_denominator > 1e-7, ttbar_numerator / ttbar_denominator, 0.)

/tmp/ipykernel_33/1322345589.py:1: RuntimeWarning: invalid value encountered in divide
  np.where(ttbar_denominator > 1e-7, ttbar_numerator / ttbar_denominator, 0.)


array([0.24161602, 0.24673043, 0.23924274, 0.19126429, 0.16562336,
       0.10668181, 0.07024908, 0.        , 0.        ])

In [92]:
mistag_rate

array([-0.16754195, -0.17641793, -0.17893146, -0.14411789, -0.12005374,
       -0.06822027, -0.04850995,  0.        ,  0.        ])

In [85]:
jetht_numerator.view().value, jetht_numerator.values()

(array([ 4.,  9., 31., 19., 18.,  4.,  1.,  0.,  0.]),
 array([ 4.,  9., 31., 19., 18.,  4.,  1.,  0.,  0.]))

In [21]:
# https://twiki.cern.ch/twiki/bin/viewauth/CMS/PdmVAnalysisSummaryTable
luminosity = {
    "2016APV": 19800.,
    "2016": 16120., #35920 - 19800
    "2017": 41530.,
    "2018": 59740.
}

ttbar_evts = {}
ttbar_evts["700to1000"] = ttbar_700to1000['cutflow']['sumw']
ttbar_evts["1000toInf"] = ttbar_1000toInf['cutflow']['sumw']

ttbar_xs = {}
ttbar_xs["700to1000"] = 831.76 * (0.09210)
ttbar_xs["1000toInf"] = 831.76 * (0.02474)
toptag_sf = 0.9
toptag_kf = 0.7

In [22]:

IOV = "2016"
ttbar_SF = {}
ttbar_SF["700to1000"] = luminosity[IOV] * ttbar_xs["700to1000"] * toptag_sf**2 * toptag_kf / ttbar_evts["1000toInf"]
ttbar_SF["1000toInf"] = luminosity[IOV] * ttbar_xs["1000toInf"] * toptag_sf**2 * toptag_kf / ttbar_evts["1000toInf"]



In [180]:
mcsv = pd.read_csv('mistag_rate_2016.csv')

In [184]:
mcsv['jetp bins'].values

array([ 400.,  500.,  600.,  800., 1000., 1500., 2000., 3000., 7000.])

In [190]:
prand = np.random.randint(400, 7500, 100)
prand

array([5983, 2987,  925, 1923, 1290, 5772,  491, 5533, 4214, 4955, 4930,
       2868, 2386, 3933,  909, 4403, 3996, 3442, 1071, 1938, 7374, 1611,
       3696, 6006, 6407, 4363, 1250, 4324, 5846, 7321, 5968, 4965, 6855,
       5421, 3851, 6620, 1348, 2263, 4225, 2234,  887, 1931, 5849, 4848,
       2878, 6120, 5092,  884,  810, 5314, 5727, 3778, 2575, 4777,  882,
       4388, 2764, 5109, 1888, 2574, 5003, 3851, 6256, 1535, 2814, 3522,
       3407, 7364, 4172, 2224, 1570,  439,  552, 5283, 4650, 4504, 1069,
       4009, 3103, 6374, 4304, 2050, 6673, 7102, 1494, 1419, 3573, 2568,
       3857, 3257, 1069, 5599, 7466, 2306, 6919, 2355, 7344, 5432, 6146,
       5269])

In [196]:
mbins = np.digitize(prand, pbins) - 1
mbins

array([7, 6, 3, 5, 4, 7, 0, 7, 7, 7, 7, 6, 6, 7, 3, 7, 7, 7, 4, 5, 8, 5,
       7, 7, 7, 7, 4, 7, 7, 8, 7, 7, 7, 7, 7, 7, 4, 6, 7, 6, 3, 5, 7, 7,
       6, 7, 7, 3, 3, 7, 7, 7, 6, 7, 3, 7, 6, 7, 5, 6, 7, 7, 7, 5, 6, 7,
       7, 8, 7, 6, 5, 0, 1, 7, 7, 7, 4, 7, 7, 7, 7, 6, 7, 8, 4, 4, 7, 6,
       7, 7, 4, 7, 8, 6, 7, 6, 8, 7, 7, 7])

In [199]:
mcsv['at0bcen'].values[mbins]

array([0.00957868, 0.02825792, 0.03811876, 0.02545634, 0.03283913,
       0.00957868, 0.05207721, 0.00957868, 0.00957868, 0.00957868,
       0.00957868, 0.02825792, 0.02825792, 0.00957868, 0.03811876,
       0.00957868, 0.00957868, 0.00957868, 0.03283913, 0.02545634,
       0.        , 0.02545634, 0.00957868, 0.00957868, 0.00957868,
       0.00957868, 0.03283913, 0.00957868, 0.00957868, 0.        ,
       0.00957868, 0.00957868, 0.00957868, 0.00957868, 0.00957868,
       0.00957868, 0.03283913, 0.02825792, 0.00957868, 0.02825792,
       0.03811876, 0.02545634, 0.00957868, 0.00957868, 0.02825792,
       0.00957868, 0.00957868, 0.03811876, 0.03811876, 0.00957868,
       0.00957868, 0.00957868, 0.02825792, 0.00957868, 0.03811876,
       0.00957868, 0.02825792, 0.00957868, 0.02545634, 0.02825792,
       0.00957868, 0.00957868, 0.00957868, 0.02545634, 0.02825792,
       0.00957868, 0.00957868, 0.        , 0.00957868, 0.02825792,
       0.02545634, 0.05207721, 0.05139537, 0.00957868, 0.00957

In [202]:
qcdfile = util.load(f'../outputs/QCD_2016.coffea')

In [209]:
qcdfile['jetmass'][{'dataset':sum, 'anacat':label_to_int_dict['2t0bcen']}].values()

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.52478069e-08, 3.04080636e-07,
       8.70998433e-07, 1.47814509e-06, 1.04100185e-06, 1.25910178e-06,
       1.41757937e-06, 1.73558173e-06, 9.84309064e-07, 1.42726135e-06,
       1.44039576e-06, 2.75280300e-07, 2.28666923e-07, 1.33773422e-07,
       7.57524391e-08, 3.11545845e-08, 1.33427762e-08, 2.48417761e-08,
       7.80186908e-09, 3.01755657e-09, 7.13076366e-09, 1.01725612e-08,
       1.38713286e-09, 3.34282503e-10, 6.76411445e-10, 5.52272194e-10,
       5.26103367e-10, 7.44152017e-10, 1.22090357e-09, 2.07592312e-10,
       2.62665140e-10, 2.21191033e-10, 4.43096820e-12, 4.67514320e-11,
       6.57216644e-11, 5.56804303e-11, 1.41271774e-10, 7.04712843e-11,
       2.13078393e-11, 5.57718435e-11])

In [215]:
qcdfile['jetmass'].axes['jetmass'].centers - qcdfile['jetmass'].axes['jetmass'].widths/2

array([  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.,
       110., 120., 130., 140., 150., 160., 170., 180., 190., 200., 210.,
       220., 230., 240., 250., 260., 270., 280., 290., 300., 310., 320.,
       330., 340., 350., 360., 370., 380., 390., 400., 410., 420., 430.,
       440., 450., 460., 470., 480., 490.])